In [12]:
import polars as pl
import pandas as pd
import numpy as np

In [109]:
# df = pl.scan_csv("data/npidata_pfile_20050523-20230409.csv")
# df = df.filter(pl.col("Provider Business Practice Location Address State Name") == "NY")
# df = df.collect()
# df.write_csv("data/ny_npi.csv")

In [82]:
df = pd.read_csv("data/ny_npi.csv")

C:\Users\yianz\AppData\Local\Temp\ipykernel_14736\3092641132.py:1: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,13,14,15,16,17,18,24,32,39,40,41,42,43,44,45,60,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,115,118,119,121,123,125,127,129,130,131,133,134,135,137,138,139,141,142,143,145,146,147,149,150,151,153,154,155,157,158,159,161,162,163,165,166,167,169,170,171,173,174,175,177,178,179,181,182,183,185,186,187,189,190,191,193,194,195,197,198,199,201,202,203,205,206,207,209,210,211,213,214,215,217,218,219,221,222,223,225,226,227,229,230,231,233,234,235,237,238,239,241,242,243,245,246,247,249,250,251,253,254,255,257,258,259,261,262,263,265,266,267,269,270,271,273,274,275,277,278,279,281,283,285,286,287,289,290,291,293,294,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.re

In [84]:
df.columns.to_list()

['NPI',
 'Entity Type Code',
 'Replacement NPI',
 'Employer Identification Number (EIN)',
 'Provider Organization Name (Legal Business Name)',
 'Provider Last Name (Legal Name)',
 'Provider First Name',
 'Provider Middle Name',
 'Provider Name Prefix Text',
 'Provider Name Suffix Text',
 'Provider Credential Text',
 'Provider Other Organization Name',
 'Provider Other Organization Name Type Code',
 'Provider Other Last Name',
 'Provider Other First Name',
 'Provider Other Middle Name',
 'Provider Other Name Prefix Text',
 'Provider Other Name Suffix Text',
 'Provider Other Credential Text',
 'Provider Other Last Name Type Code',
 'Provider First Line Business Mailing Address',
 'Provider Second Line Business Mailing Address',
 'Provider Business Mailing Address City Name',
 'Provider Business Mailing Address State Name',
 'Provider Business Mailing Address Postal Code',
 'Provider Business Mailing Address Country Code (If outside U.S.)',
 'Provider Business Mailing Address Telephone Nu

In [91]:
zipcodes = df["Provider Business Practice Location Address Postal Code"].astype(str).str[0:5]
zipcodes = np.where(zipcodes.str.isnumeric(), zipcodes, 0)

In [92]:
df["zipcode"] = zipcodes.astype(int)

In [97]:
df_nyc = df[((df["zipcode"] >= 10001) & (df["zipcode"] <= 10499)) | ((df["zipcode"] >= 11001) & (df["zipcode"] <= 11499))]

In [93]:
urgent = ["261QU0200X", "261QE0002X", "207PE0004X", "207PP0204X", "2080P0204X", "146L00000X"]

In [104]:
def check_valid(entry):
    base = "Healthcare Provider Taxonomy Code_"
    for i in range(1, 16):
        if entry[base + str(i)] in urgent:
            return True
    return False

In [106]:
df_urgent = df_nyc[df_nyc.apply(lambda x: check_valid(x), axis=1)]

In [110]:
df_urgent

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,...,Healthcare Provider Taxonomy Group_8,Healthcare Provider Taxonomy Group_9,Healthcare Provider Taxonomy Group_10,Healthcare Provider Taxonomy Group_11,Healthcare Provider Taxonomy Group_12,Healthcare Provider Taxonomy Group_13,Healthcare Provider Taxonomy Group_14,Healthcare Provider Taxonomy Group_15,Certification Date,zipcode
1143,1396747135,1,NaN,NaN,NaN,TEJANI,NOORUDDIN,RAHMATULLAH,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11203
1457,1821081506,1,NaN,NaN,NaN,GREKIN,PETER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10010
1459,1437142130,1,NaN,NaN,NaN,KWON,NANCY,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10016
1725,1255326336,1,NaN,NaN,NaN,DAVE,AMI,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10016
1745,1720073802,1,NaN,NaN,NaN,DILL,CURT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557452,1679185904,2,NaN,<UNAVAIL>,"MOBILE MEDICAL HEALTHCARE, P.C.",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/27/2023,10001
558040,1689106775,1,NaN,NaN,NaN,PIZZITOLA,GINA,NaN,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/28/2023,11030
559518,1982714622,1,NaN,NaN,NaN,GENTRY,DAVID,WILLIAM,MR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/04/2023,10016
560271,1750810750,1,NaN,NaN,NaN,PROPER,CAITLIN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/06/2023,10467


In [112]:
df_urgent.iloc[5].to_dict()

{'NPI': 1811987936,
 'Entity Type Code': 1,
 'Replacement NPI': nan,
 'Employer Identification Number (EIN)': nan,
 'Provider Organization Name (Legal Business Name)': nan,
 'Provider Last Name (Legal Name)': 'NADERI',
 'Provider First Name': 'SASSAN',
 'Provider Middle Name': nan,
 'Provider Name Prefix Text': nan,
 'Provider Name Suffix Text': nan,
 'Provider Credential Text': 'M.D.',
 'Provider Other Organization Name': nan,
 'Provider Other Organization Name Type Code': nan,
 'Provider Other Last Name': nan,
 'Provider Other First Name': nan,
 'Provider Other Middle Name': nan,
 'Provider Other Name Prefix Text': nan,
 'Provider Other Name Suffix Text': nan,
 'Provider Other Credential Text': nan,
 'Provider Other Last Name Type Code': nan,
 'Provider First Line Business Mailing Address': '550 1ST AVE',
 'Provider Second Line Business Mailing Address': nan,
 'Provider Business Mailing Address City Name': 'NEW YORK',
 'Provider Business Mailing Address State Name': 'NY',
 'Provider 